In [ ]:
library(readr)
library(tidyr)
library(stringr)
library(dplyr)
library(lubridate)
library(USAboundaries)
library(sf)

In [ ]:
fp      <- '/pool001/mfzhao/'
keys    <- read_csv(str_c(fp, 'PROCESSED_DATA/keys.csv'))
policy  <- read_csv(str_c(fp, 'PROCESSED_DATA/policy.csv'))
df      <- read_csv(str_c(fp, 'safegraph/dyadic2020.csv'))
us_pop  <- read_csv(str_c(fp, 'geo_data/cc-est2018-alldata.csv'))
panel   <- read_rds(str_c(fp, 'PROCESSED_DATA/panel_pre_xgr.RDS'))
usm     <- read_csv(str_c(fp, 'safegraph/us_mobility.csv'), 
                    col_types = cols(
                        mhdt = col_double(),
                        mnhdt = col_double(),
                        mdtfh = col_double()))

In [ ]:
county_sf <- us_counties()

as.data.frame(county_sf) %>%
    select(-geometry) %>%
    mutate(L3 = row_number(),
           key = str_c(statefp, countyfp)) %>%
    left_join(as.data.frame(st_coordinates(county_sf))) %>%
    select(key, X, Y) -> coords

coords %>%
    inner_join(coords, by = c('X', 'Y')) %>%
    filter(key.x != key.y) %>%
    select(origin_county = key.x,
           destination_county = key.y) %>%
    distinct() %>%
    mutate(bordering = 1) %>%
    arrange(origin_county, destination_county) -> bordering

county_sf %>%
    st_centroid() %>%
    as.data.frame() %>%
    mutate(key = str_c(statefp, countyfp)) %>%
    select(key, geometry) %>%
    arrange(key) -> centroids

distm <- data.frame(st_distance(centroids$geometry, centroids$geometry))
colnames(distm) <- centroids$key

distm %>%
    mutate(origin_county = centroids$key) %>%
    gather(key = 'destination_county', value = 'dist', -origin_county) %>%
    mutate(dist = as.numeric(dist)/1000) %>% 
    inner_join(keys, by = c('origin_county' = 'key')) %>%
    inner_join(keys, by = c('destination_county' = 'key') )-> distm

us_pop %>%
    unite('key', STATE, COUNTY, sep = '') %>%
    filter(YEAR == 11, 
           AGEGRP==0) %>% 
    group_by(key) %>%
    summarize(n = sum(TOT_POP)) -> us_pop

usm %>%
    rename(key = origin_county) %>%
    filter(date >= as.Date('2020-01-01'), date < as.Date('2020-07-01')) %>%
    inner_join(keys) %>%
    select(date, key, device_count) -> dc

In [ ]:
df %>%
    filter(date >= as.Date('2020-01-01'), date < as.Date('2020-07-01')) %>%
    mutate(origin_county      = str_pad(origin_county, 5, pad = '0'),
           destination_county = str_pad(destination_county, 5, pad = '0')) %>%
    inner_join(keys, by = c('origin_county' = 'key')) %>%
    inner_join(keys, by = c('destination_county' = 'key')) %>%
    left_join(dc, by = c('date', 'origin_county' = 'key')) %>%
    left_join(bordering) %>%
    mutate(pod = n/device_count,
           bordering = replace_na(bordering, 0)) -> df

df %>%
    group_by(origin_county, destination_county) %>%
    summarize(n = sum(n), days = n()) -> dyad_stats

In [ ]:
dyad_stats %>%
    group_by(days) %>%
    summarize(n = sum(n)) %>%
    arrange(desc(n)) %>%
    mutate(ecdf = cumsum(n)/sum(n)) %>%
    head()

In [ ]:
dyad_stats %>%
    filter(origin_county != destination_county) %>%
    group_by(days) %>%
    summarize(n = sum(n)) %>%
    arrange(desc(n)) %>%
    mutate(ecdf = cumsum(n)/sum(n)) %>%
    head()

In [7]:
panel %>%
    ungroup() %>%
    select(date, key, PRCP, TMAX, matches('^prcp..$'), matches('^tmax..$')) -> weather

oweather <- weather
dweather <- weather

owc <- str_c('o', colnames(oweather))
owc[1:2] <- c('date', 'origin_county')
colnames(oweather) <- owc

dwc <- str_c('d', colnames(dweather))
dwc[1:2] <- c('date', 'destination_county')
colnames(dweather) <- dwc 

In [8]:
df %>%
    inner_join(dyad_stats %>% filter(days == max(days)) %>% select(origin_county, destination_county)) %>%
    inner_join(policy %>% 
               select(date,
                      origin_county = key,
                      op1 = p1sdp,
                      op2 = p2shp,
                      op3 = p3rop)) %>%
    inner_join(policy %>% 
               select(date,
                      destination_county = key,
                      dp1 = p1sdp,
                      dp2 = p2shp,
                      dp3 = p3rop)) %>% 
    filter(origin_county != destination_county) -> df

Joining, by = c("origin_county", "destination_county")

Joining, by = c("date", "origin_county")

Joining, by = c("date", "destination_county")



In [9]:
panel %>%
    ungroup() %>%
    select(key, date, matches('new')) %>%
    mutate(as_newcases  = asinh(newcases),
           as_newdeaths = asinh(newdeaths),
           as_stnewcases  = asinh(stnewcases),
           as_stnewdeaths = asinh(stnewdeaths)) -> cd

ocd <- cd
dcd <- cd

ocdc <- str_c('o', colnames(ocd))
ocdc[1:2] <- c('origin_county', 'date')
colnames(ocd) <- ocdc

dcdc <- str_c('d', colnames(dcd))
dcdc[1:2] <- c('destination_county', 'date')
colnames(dcd) <- dcdc

In [10]:
df %>%
    rename(ndotd = n, pdotd = pod) %>%
    mutate(dyad = str_c(origin_county, '->', destination_county),
           origin_cluster = str_sub(origin_county, 1, 2), 
           destination_cluster = str_sub(destination_county, 1, 2)) %>%
    left_join(us_pop, by = c('origin_county' = 'key')) %>%
    left_join(distm) %>%
    select(date, dyad, origin_county, destination_county, 
           origin_cluster, destination_cluster, n, bordering, ndotd, pdotd, dist, 
           op1, op2, op3, 
           dp1, dp2, dp3) %>%
    left_join(oweather) %>%
    left_join(dweather) %>%
    left_join(ocd) %>%
    left_join(dcd) %>%
    select(-origin_county, -destination_county) -> df

Joining, by = c("origin_county", "destination_county")

Joining, by = c("date", "origin_county")

Joining, by = c("date", "destination_county")

Joining, by = c("date", "origin_county")

Joining, by = c("date", "destination_county")



In [11]:
df %>%
    select(-op1, -op2, -op3, -dp1, -dp2, -dp3) %>%
    bind_cols(as.data.frame(model.matrix(n ~ 0 + (op1 + op2 + op3) * (dp1 + dp2 + dp3), df))) -> df

In [12]:
data.table::fwrite(df, str_c(fp, 'PROCESSED_DATA/dyadic_mvmt_2020.csv'))